Cloning into 'charsiu'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 322 (delta 158), reused 159 (delta 55), pack-reused 0
Receiving objects: 100% (322/322), 511.27 KiB | 12.47 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Your branch is up to date with 'origin/development'.
/root


In [24]:
import sys
import torch
from itertools import groupby
from datasets import load_dataset
import matplotlib.pyplot as plt


from charsiu.src.Charsiu import Wav2Vec2ForFrameClassification, CharsiuPreprocessor_en, charsiu_forced_aligner, charsiu_chain_attention_aligner, charsiu_chain_forced_aligner, charsiu_predictive_aligner


In [25]:
# download timit
timit = load_dataset('timit_asr', data_dir='/home/tomi/Documents/tesis_speechRate/timit')


/home/tomi/.local/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for timit_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/timit_asr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [26]:
# load data
sample = timit['train'][1]
text = sample['text']
audio_path = sample['file']
print('Text transcription:%s'%(text))
print('Audio path: %s'%audio_path)

Text transcription:Don't ask me to carry an oily rag like that.
Audio path: /home/tomi/Documents/tesis_speechRate/timit/data/TRAIN/DR1/FCJF0/SA2.WAV


Phone recognizer + Neural Forced Alignment

In [31]:
# load model
charsiu = charsiu_chain_attention_aligner(aligner='charsiu/en_w2v2_fs_10ms',recognizer='charsiu/en_w2v2_ctc_libris_and_cv')


Some weights of the model checkpoint at charsiu/en_w2v2_fs_10ms were not used when initializing Wav2Vec2ForAttentionAlignment: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForAttentionAlignment from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForAttentionAlignment from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForAttentionAlignment were not initialized from the model checkpoint at charsiu/en_w2v2_fs_10ms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weig

In [6]:
alignment = charsiu.align(audio=audio_path)

/home/tomi/Documents/tesis_speechRate/charsiu/src/Charsiu.py:372: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(audio).float().unsqueeze(0).to(self.device)


In [8]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

[(0.0, 0.17, '[SIL]'), (0.17, 0.2, '[UNK]'), (0.2, 0.31, 'S'), (0.31, 0.35, '[UNK]'), (0.35, 0.38, 'S'), (0.38, 0.41, '[UNK]'), (0.41, 0.44, 'L'), (0.44, 0.56, '[UNK]'), (0.56, 0.63, 'S'), (0.63, 0.93, '[UNK]'), (0.93, 1.01, 'K'), (1.01, 1.1, 'S'), (1.1, 1.24, '[UNK]'), (1.24, 1.27, 'L'), (1.27, 1.33, '[UNK]'), (1.33, 1.38, 'N'), (1.38, 1.42, 'D'), (1.42, 1.47, 'G'), (1.47, 1.52, 'R'), (1.52, 1.57, '[UNK]'), (1.57, 1.61, 'Y'), (1.61, 1.69, 'S'), (1.69, 1.71, '[UNK]'), (1.71, 1.79, 'Y'), (1.79, 1.88, 'W'), (1.88, 2.0, '[UNK]'), (2.0, 2.08, 'S'), (2.08, 2.1, '[UNK]'), (2.1, 2.18, 'W'), (2.18, 2.26, '[UNK]'), (2.26, 2.31, 'R'), (2.31, 2.46, '[UNK]'), (2.46, 2.52, 'L'), (2.52, 2.66, 'Y'), (2.66, 2.67, '[UNK]'), (2.67, 2.69, 'L'), (2.69, 2.8, '[UNK]'), (2.8, 2.9, '[SIL]')]

 Ground Truth 

[(0.0, 0.190625, 'h#'), (0.190625, 0.2849375, 'sh'), (0.2849375, 0.3576875, 'ix'), (0.3576875, 0.415125, 'hv'), (0.415125, 0.54825, 'eh'), (0.54825, 0.574375, 'dcl'), (0.574375, 0.6460625, 'jh'), (0.64606

In [9]:
charsiu.serve(audio=audio_path, save_to='sample.TextGrid')

/home/tomi/Documents/tesis_speechRate/charsiu/src/Charsiu.py:372: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(audio).float().unsqueeze(0).to(self.device)


Alignment output has been saved to sample.TextGrid


In [28]:
# load model
charsiu = charsiu_chain_forced_aligner(aligner='charsiu/en_w2v2_fc_10ms',recognizer='charsiu/en_w2v2_ctc_libris_and_cv')

Some weights of the model checkpoint at charsiu/en_w2v2_fc_10ms were not used when initializing Wav2Vec2ForFrameClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForFrameClassification were not initialized from the model checkpoint at charsiu/en_w2v2_fc_10ms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.

In [11]:
alignment = charsiu.align(audio=audio_path)

/home/tomi/Documents/tesis_speechRate/charsiu/src/Charsiu.py:457: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(audio).float().unsqueeze(0).to(self.device)


In [12]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

[(0.0, 0.23, '[SIL]'), (0.23, 0.27, '[UNK]'), (0.27, 0.28, 'S'), (0.28, 0.29, '[UNK]'), (0.29, 0.3, 'L'), (0.3, 0.31, '[UNK]'), (0.31, 0.32, 'S'), (0.32, 0.42, '[UNK]'), (0.42, 0.43, 'Y'), (0.43, 0.51, '[UNK]'), (0.51, 0.65, 'D'), (0.65, 0.7, '[UNK]'), (0.7, 0.84, 'D'), (0.84, 0.9, '[UNK]'), (0.9, 1.01, 'K'), (1.01, 1.14, 'S'), (1.14, 1.23, '[UNK]'), (1.23, 1.24, 'L'), (1.24, 1.26, '[UNK]'), (1.26, 1.37, 'N'), (1.37, 1.4, 'D'), (1.4, 1.44, 'G'), (1.44, 1.53, 'R'), (1.53, 1.57, '[UNK]'), (1.57, 1.58, 'Y'), (1.58, 1.69, 'S'), (1.69, 1.72, '[UNK]'), (1.72, 1.73, 'Y'), (1.73, 1.89, 'W'), (1.89, 1.97, '[UNK]'), (1.97, 1.98, 'S'), (1.98, 2.05, '[UNK]'), (2.05, 2.15, 'W'), (2.15, 2.17, '[UNK]'), (2.17, 2.31, 'R'), (2.31, 2.33, '[UNK]'), (2.33, 2.51, 'L'), (2.51, 2.67, 'Y'), (2.67, 2.69, '[UNK]'), (2.69, 2.81, 'R'), (2.81, 2.82, '[UNK]'), (2.82, 2.83, 'S'), (2.83, 2.84, '[UNK]'), (2.84, 2.85, 'L'), (2.85, 2.86, '[UNK]'), (2.86, 2.9, '[SIL]')]

 Ground Truth 

[(0.0, 0.190625, 'h#'), (0.190625,

In [13]:
charsiu.serve(audio=audio_path, save_to='sample.TextGrid')

Alignment output has been saved to sample.TextGrid


Direct inference with frame classification model

In [14]:
charsiu = charsiu_predictive_aligner(aligner='charsiu/en_w2v2_fc_10ms')

Some weights of the model checkpoint at charsiu/en_w2v2_fc_10ms were not used when initializing Wav2Vec2ForFrameClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForFrameClassification were not initialized from the model checkpoint at charsiu/en_w2v2_fc_10ms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.

In [18]:
alignment = charsiu.align(audio=audio_path)

In [19]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

[(0.0, 0.1, '[SIL]'), (0.1, 0.19, 'D'), (0.19, 0.2, 'OW'), (0.2, 0.24, 'AH'), (0.24, 0.29, 'N'), (0.29, 0.43, 'AE'), (0.43, 0.48, 'S'), (0.48, 0.52, 'K'), (0.52, 0.55, 'M'), (0.55, 0.59, 'IY'), (0.59, 0.63, 'T'), (0.63, 0.67, 'AH'), (0.67, 0.77, 'K'), (0.77, 0.86, 'EH'), (0.86, 0.91, 'R'), (0.91, 0.94, 'IY'), (0.94, 0.97, 'AH'), (0.97, 1.03, 'N'), (1.03, 1.16, 'OY'), (1.16, 1.18, 'IH'), (1.18, 1.26, 'L'), (1.26, 1.33, 'AH'), (1.33, 1.39, 'R'), (1.39, 1.54, 'AE'), (1.54, 1.58, 'G'), (1.58, 1.62, 'L'), (1.62, 1.69, 'AY'), (1.69, 1.77, 'K'), (1.77, 1.84, 'DH'), (1.84, 2.01, 'AE'), (2.01, 2.04, 'T'), (2.04, 2.14, '[SIL]')]

 Ground Truth 

[(0.0, 0.14125, 'h#'), (0.14125, 0.170625, 'd'), (0.170625, 0.2575, 'uh'), (0.2575, 0.2875, 'n'), (0.2875, 0.429, 'ae'), (0.429, 0.495, 's'), (0.495, 0.516875, 'kcl'), (0.516875, 0.54, 'k'), (0.54, 0.5535, 'm'), (0.5535, 0.595, 'ix'), (0.595, 0.6225, 'dx'), (0.6225, 0.671, 'ix'), (0.671, 0.71875, 'kcl'), (0.71875, 0.78375, 'k'), (0.78375, 0.8459375, 'eh'

In [29]:
charsiu.serve(audio=audio_path, save_to='sample.TextGrid')

Alignment output has been saved to sample.TextGrid
